In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
import time
import json

url = "https://platform.stratascratch.com/visualizations?code_type=2&page_size=100&order_field=difficulty"

In [13]:
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.page_load_strategy = 'none'
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def scraping_data(url, func):
    driver = setup_driver()
    driver.implicitly_wait(4)
    driver.get(url)
    time.sleep(4)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    if func == 'get_list_data':
        getting_list_data(soup)
    elif func == 'get_data_info':
        getting_data_info(url, soup)

    driver.quit()

def getting_list_data(soup):
    data = []
    for list in soup.find_all('a', 'group contents'):
        details = list.find_all('div')
        info = {}
        info['link'] = 'https://platform.stratascratch.com' + list.get('href')
        info['difficulty'] = details[1].find('span').getText()
        info['id'] = details[2].getText()
        info['title'] = details[3].getText()
        data.append(info)

    json_dump('visualization_list.json', data)  

def getting_data_info(link,soup):
    questionare = {}
    questionare['link'] = link
    questionare['title'] = soup.find('h1','QuestionMetadata__h1').getText()
    
    question_des_list = soup.find_all('div', 'QuestionMetadata__metadata')
    question_meta =  question_des_list[0].find_all('span')
    questionare['difficulty'] = question_meta[0].getText()
    questionare['id'] = question_meta[1].getText()
    questionare['type'] = question_meta[2].getText()
    questionare['analysis'] = question_meta[3].getText()
    questionare['data_type'] = question_meta[4].getText()
    questionare['description'] = soup.find('div','QuestionMetadata__question').find('p').getText()
    questionare['data'] = {}
    questionare['solution'] = ""

    file_title = 'visualization_notebook/' + questionare['title'].replace(' ', '-') + '.json'
    json_dump(file_title, questionare)

def json_dump(file, data):
    with open(file, 'w+') as f:
        json.dump(data, f)

#scraping_data(url,'get_list_data')

In [32]:
visual_list = []
with open('visualization_list.json', 'r+') as list_viz:
    for viz in json.load(list_viz):
        scraping_data(viz['link'],'get_data_info')

In [45]:
source_code = """import matplotlib.pyplot as plt
import numpy as np

# Define colors for each appliance
colors = ['mediumblue', 'darkorange', 'limegreen']

# Prepare data
labels = df['appliance']
energy = df['energy_consumption']

# Number of variables
num_vars = len(labels)

# Compute the angle each bar is centered on:
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# Complete the loop to close the circular graph
energy = np.concatenate((energy, [energy[0]]))
angles += angles[:1]

# Draw plot
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))

# Draw bars
for i in range(num_vars):
    ax.bar(angles[i], energy[i], color=colors[i], width=0.4, label=labels[i], alpha=0.7)

# Connect the last bar to the first to close the shape
ax.plot(angles, energy, color='black', linewidth=2)  # Outline

# Set the labels for each appliance at the appropriate angle
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels)

# Title
plt.title('Energy Consumption Rates of Various Appliances', size=15, color='black', y=1.1)
plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

plt.show()
"""

data = {
    "source_code": source_code
}

json_string = json.dumps(data, separators=(',', ':'))

print(json_string)

{"source_code":"import matplotlib.pyplot as plt\nimport numpy as np\n\n# Define colors for each appliance\ncolors = ['mediumblue', 'darkorange', 'limegreen']\n\n# Prepare data\nlabels = df['appliance']\nenergy = df['energy_consumption']\n\n# Number of variables\nnum_vars = len(labels)\n\n# Compute the angle each bar is centered on:\nangles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()\n\n# Complete the loop to close the circular graph\nenergy = np.concatenate((energy, [energy[0]]))\nangles += angles[:1]\n\n# Draw plot\nfig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))\n\n# Draw bars\nfor i in range(num_vars):\n    ax.bar(angles[i], energy[i], color=colors[i], width=0.4, label=labels[i], alpha=0.7)\n\n# Connect the last bar to the first to close the shape\nax.plot(angles, energy, color='black', linewidth=2)  # Outline\n\n# Set the labels for each appliance at the appropriate angle\nax.set_xticks(angles[:-1])\nax.set_xticklabels(labels)\n\n# Title\nplt.

In [46]:
sc = ''' 
<table class="ResultsTable__table"><thead><tr class="ResultsTable__header-row"><th class="ResultsTable__header-cell">appliance</th><th class="ResultsTable__header-cell">energy_consumption</th></tr></thead><tbody><tr class="ResultsTable__row "><td class="ResultsTable__cell">Refrigerator</td><td class="ResultsTable__cell">500</td></tr><tr class="ResultsTable__row "><td class="ResultsTable__cell">Heater</td><td class="ResultsTable__cell">800</td></tr><tr class="ResultsTable__row "><td class="ResultsTable__cell">Air Conditioner</td><td class="ResultsTable__cell">600</td></tr></tbody></table>
 '''

soup = BeautifulSoup(sc, "html.parser")
headers = [headText.getText() for headText in soup.find_all('th', 'ResultsTable__header-cell')]
data = []
for row in soup.find_all('tr','ResultsTable__row'):
    cell = []
    for cells in row.find_all('td', 'ResultsTable__cell'):
        cell.append(cells.getText())

    data.append(cell)

json_data = [dict(zip(headers, row)) for row in data]
json_output = json.dumps(json_data, indent=4)

print(json_output)


[
    {
        "appliance": "Refrigerator",
        "energy_consumption": "500"
    },
    {
        "appliance": "Heater",
        "energy_consumption": "800"
    },
    {
        "appliance": "Air Conditioner",
        "energy_consumption": "600"
    }
]
